***Practical Data Science Assignment 2***

PANAGIOTIS KAPSALIS DS3516005

In [1]:
import pandas as pd
import scipy
from yahoo_finance import Share
from pprint import pprint
from scipy import stats
import numpy as np
import scikits.bootstrap as bootstrap
import time 

First we find the tickers for the firms that we will investigate. To calculate the returns for each firm between 21-10-2008 and 08-12-2008 we must find the open  stock value and the close stock value for each day. We will use yahoo finance to get the necessary information about the daily stock values. There is the python package yahoo-finance, which we must download and use his functions. Specifically we use the functions Share and get_historical. Share function takes as argument the ticker of each firm and the function get_historical takes as arguments the dates, and yahoo_finance wil return the requested information about the stocks of thiw firm. We compute the daily returns for each firm using the following formula

\begin{equation*}
returns  = (Closing\;stock\;price - Opening\;stock\;price)/(Opening\;stock\;price)
\end{equation*}

We must note that as Open we mean Opening stock price and as Close we mean the Closing stock price. The function get_historical returns a list of dictionaries, each dictionaty contains the necassary information about each day, which was in days between the date set that function get_historical takes them as arguments.

In [2]:
firms = ['BK','BFED','BLK','CARV','C','CME','EVR','FII','FIG','GBL','LAZ','NTRS','BPOP','PFS','STT','BX','NDAQ']

In [3]:
info = pd.DataFrame()
for firm in firms:
    ff = Share(firm)
    finance = ff.get_historical('2008-11-21','2008-12-08')
    time.sleep(1)
    temp = pd.DataFrame(finance)
    firm_df = temp[['Date','Open','Close']]
    firm_df['Open'] = pd.to_numeric(firm_df['Open'])
    firm_df['Close'] = pd.to_numeric(firm_df['Close'])
    string = 'returns'+'_'+firm
    firm_df[string] = ((firm_df['Close'] - firm_df['Open'])/firm_df['Open'])*100
    temp1 = firm_df[[string,'Date']]
    temp1 = temp1.append({string:sum(temp1[string]), 'Date':'Cumulative'}, ignore_index=True)
    temp1 = temp1.set_index('Date')
    info = pd.concat([info,temp1], axis = 1,)
info["returns_BFED"]['2008-11-25']=0.0000
info["returns_BFED"]['2008-12-08']=0.0000
pprint(info)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

            returns_BK  returns_BFED  returns_BLK  returns_CARV  returns_C  \
2008-11-21    2.743539     -0.123762     6.879972      0.000000 -30.698529   
2008-11-24   20.111732      0.001252     3.706013     -0.153854  -2.777778   
2008-11-25   -8.122301      0.000000    -2.035541      2.763403  -4.851330   
2008-11-26    3.811497     -0.121359     8.697170      1.633385  14.262561   
2008-11-28    1.104418      5.454545     1.789473      0.000000  11.275168   
2008-12-01  -11.533193     -0.118906   -11.227198      0.000000 -18.354430   
2008-12-02    5.112312     -0.544054     4.431136      1.818182   4.335260   
2008-12-03   12.338648      0.000000    11.743122     -0.178262  13.828239   
2008-12-04   -1.211485      8.387097    -2.861942      0.000000  -2.759527   
2008-12-05    6.393508      0.000000    12.247398      0.000000   6.344828   
2008-12-08   -2.782166      0.000000     6.278375      6.463878   2.916160   
Cumulative   27.966509     12.934813    39.647978     12.346732 

We must note that the output is a dataframe which contains as columns the returns for each firm, the dataframe, also, has as indexes the days that we will investigate. Furthermore the last row contains the cumulative returns for each firm for this ten days interval 

In [4]:
info['Connected'] = info.mean(axis=1)
pprint(info)

            returns_BK  returns_BFED  returns_BLK  returns_CARV  returns_C  \
2008-11-21    2.743539     -0.123762     6.879972      0.000000 -30.698529   
2008-11-24   20.111732      0.001252     3.706013     -0.153854  -2.777778   
2008-11-25   -8.122301      0.000000    -2.035541      2.763403  -4.851330   
2008-11-26    3.811497     -0.121359     8.697170      1.633385  14.262561   
2008-11-28    1.104418      5.454545     1.789473      0.000000  11.275168   
2008-12-01  -11.533193     -0.118906   -11.227198      0.000000 -18.354430   
2008-12-02    5.112312     -0.544054     4.431136      1.818182   4.335260   
2008-12-03   12.338648      0.000000    11.743122     -0.178262  13.828239   
2008-12-04   -1.211485      8.387097    -2.861942      0.000000  -2.759527   
2008-12-05    6.393508      0.000000    12.247398      0.000000   6.344828   
2008-12-08   -2.782166      0.000000     6.278375      6.463878   2.916160   
Cumulative   27.966509     12.934813    39.647978     12.346732 

Then we add 3 columns still on our table. The first is the column Connected, which contains for each day the mean of daily returns. Also calculated the mean for commulative values. The second column is the Non-Connected, which contains the values of S&P 500 index for this 10-day interval. The column Difference contains the difference between columns Connected and Non-Connected

In [6]:
sp500 = Share('^GSPC')
hist_data = sp500.get_historical('2008-11-21','2008-12-08')
sp500_df = pd.DataFrame(hist_data)
sp500_df = sp500_df[['Open','Close','Date']]
sp500_df['Open'] = pd.to_numeric(sp500_df['Open'])
sp500_df['Close'] = pd.to_numeric(sp500_df['Close'])
sp500_df['returns'] = ((sp500_df['Close'] - sp500_df['Open'])/sp500_df['Open'])*100
sp500_df = sp500_df[['Date','returns']]
sp500_df = sp500_df.append({'returns':sum(sp500_df['returns']), 'Date':'Cumulative'}, ignore_index=True).set_index('Date')
info['Non-Connected'] = sp500_df['returns']
info['Difference'] = info['Connected'] - info['Non-Connected']
pprint(info)

            returns_BK  returns_BFED  returns_BLK  returns_CARV  returns_C  \
2008-11-21    2.743539     -0.123762     6.879972      0.000000 -30.698529   
2008-11-24   20.111732      0.001252     3.706013     -0.153854  -2.777778   
2008-11-25   -8.122301      0.000000    -2.035541      2.763403  -4.851330   
2008-11-26    3.811497     -0.121359     8.697170      1.633385  14.262561   
2008-11-28    1.104418      5.454545     1.789473      0.000000  11.275168   
2008-12-01  -11.533193     -0.118906   -11.227198      0.000000 -18.354430   
2008-12-02    5.112312     -0.544054     4.431136      1.818182   4.335260   
2008-12-03   12.338648      0.000000    11.743122     -0.178262  13.828239   
2008-12-04   -1.211485      8.387097    -2.861942      0.000000  -2.759527   
2008-12-05    6.393508      0.000000    12.247398      0.000000   6.344828   
2008-12-08   -2.782166      0.000000     6.278375      6.463878   2.916160   
Cumulative   27.966509     12.934813    39.647978     12.346732 

In [7]:
event_days = ['2008-11-21','2008-11-24','2008-11-25','2008-11-26','2008-11-28','2008-12-01','2008-12-02','2008-12-03','2008-12-04','2008-12-05','2008-12-08','Cumulative']

Now we want to examine the daily returns of firms substracted from daily S&P 500 index, more specifically we want to check if these daily samples have mean (expected value), equal to zero. Before we use the t-test, we must check the normality of our data. In the code above, we implement the Kolmogorov-Smirnov test for our standardized sample, because t-test is used only for samples that follow the Normal Distribution. In our case we can see that some daily samples follow Normal and some others don't follow normal distribution. So in this case we will use a non-parametric test. We choose to use Bootstrap test.  

In [9]:
for day in event_days:
    stocks = info.loc[day,'returns_BK':'returns_NDAQ']
    day_sp500 = info.loc[day,'Non-Connected']
    sample = stocks - day_sp500
    standrdized = (sample - np.mean(sample))/np.std(sample)
    pprint(day)
    test = stats.mstats.normaltest(standrdized.astype(np.float64))
    pprint(test)
    if test[1]>0.05:
        print("Null Hypothesis is not rejected, p-value:",test[1],">0.05")
    else:
        print("Null Hypothesis is rejected, p-value:",test[1],"<0.05")
    
    

'2008-11-21'
NormaltestResult(statistic=16.899414929129016, pvalue=0.00021396299797219129)
Null Hypothesis is rejected, p-value: 0.000213962997972 <0.05
'2008-11-24'
NormaltestResult(statistic=10.3763887638556, pvalue=0.0055820768196788283)
Null Hypothesis is rejected, p-value: 0.00558207681968 <0.05
'2008-11-25'
NormaltestResult(statistic=0.27141347101924018, pvalue=0.87309864377619306)
Null Hypothesis is not rejected, p-value: 0.873098643776 >0.05
'2008-11-26'
NormaltestResult(statistic=2.2950934393757985, pvalue=0.3174145217636839)
Null Hypothesis is not rejected, p-value: 0.317414521764 >0.05
'2008-11-28'
NormaltestResult(statistic=3.3296354694042409, pvalue=0.18922514401632329)
Null Hypothesis is not rejected, p-value: 0.189225144016 >0.05
'2008-12-01'
NormaltestResult(statistic=3.3761748749432496, pvalue=0.18487276681214448)
Null Hypothesis is not rejected, p-value: 0.184872766812 >0.05
'2008-12-02'
NormaltestResult(statistic=0.038776895508061515, pvalue=0.98079829933954754)
Null

C:\Users\User\Anaconda3\lib\site-packages\scipy\stats\mstats_basic.py:2231: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  np.min(n))


Bootstrap test is a non-centralized test, takes as argument the daily returns of the firms substracted by the daily S&P 500 index. Bootstrap test takes our sample and find the mean of this sample, and has as output an interval which says that the population mean must be in this interval. This interval is the margin of error from what the mean of our sample is. In the code bellow we check if the daily difference, is in this interval

In [17]:
for day in event_days:
    stocks = info.loc[day,'returns_BK':'returns_NDAQ']
    diff = info.loc[day,'Difference']
    day_sp500 = info.loc[day,'Non-Connected']
    sample = stocks - day_sp500
    pprint(day)
    CIs = bootstrap.ci(data=sample.astype(np.float64), statfunction=scipy.mean)
    print("Low:",CIs[0]," High:",CIs[1]," mean difference is between CIs:",(CIs[0]<diff) & (diff<CIs[1]))
    

'2008-11-21'
Low: -12.6321315523  High: -2.52269777159  mean difference is between CIs: True
'2008-11-24'
Low: -1.12582856467  High: 6.77710610635  mean difference is between CIs: True
'2008-11-25'
Low: -3.48155437534  High: -0.0306249816983  mean difference is between CIs: True
'2008-11-26'
Low: 3.07876927  High: 10.1933488011  mean difference is between CIs: True
'2008-11-28'
Low: -0.73040374923  High: 2.93954475268  mean difference is between CIs: True
'2008-12-01'
Low: -3.78881353277  High: 1.54824305067  mean difference is between CIs: True
'2008-12-02'
Low: -2.54365599488  High: 1.10253717279  mean difference is between CIs: True
'2008-12-03'
Low: -3.48728925182  High: 5.54232947266  mean difference is between CIs: True
'2008-12-04'
Low: 2.00080477675  High: 6.23126803997  mean difference is between CIs: True
'2008-12-05'
Low: -1.10498014552  High: 4.63653330785  mean difference is between CIs: True
'2008-12-08'
Low: -1.21862878705  High: 2.11125476635  mean difference is between

As we can see from above we can not reject the Null Hypothesis because for each day, the daily Difference is in the Confidence Interval. 
Furthermore we must note that to use Bootstrap test in python we installed **scikits.bootstrap** package